<a href="https://colab.research.google.com/github/ChinaYiqun/3y/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装库

In [ ]:
!pip install efficientnet
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat 
!tar zxvf 102flowers.tgz -C ./

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import shutil
from keras.preprocessing.image import ImageDataGenerator
import efficientnet.keras as efn 
conv_base = efn.EfficientNetB1(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
import efficientnet.keras as efn 
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D
from keras.models import Sequential, Model, load_model
from scipy.io import loadmat
data_m= loadmat("imagelabels.mat")
labels = data_m['labels'].tolist()[0]



In [3]:
def mkdir_for_keras(dirname):
  os.mkdir(dirname)
  for i in range(1,103):
    file_name="{}_label".format(i)
    os.makedirs(dirname + '/' + file_name)
def move2dir(src,trainfolder,validfoder,labels,split = 0.2):
  for folderName, subfolders, filenames in os.walk(src):
    for filename in filenames:
      if '.jpg' in filename:
        file_id = int(filename.split('_')[1].split('.')[0]) -1
        file_class = str(labels[file_id]) + '_label'
        #print(np.random.random())
      try:
        p = np.random.random()
        #print(p)
        #print(folderName+ '/'+ filename,validfoder+ '/'+file_class +'/'+ filename)
        
        if p < split:
          # 放入验证集
          #print(folderName+ '/'+ filename,validfoder+ '/'+file_class +'/'+ filename)
          shutil.move(folderName+ '/'+ filename,validfoder+ '/'+file_class +'/'+ filename)
          
          pass
        else:
          # 放入训练集
          shutil.move(folderName+ '/'+ filename,trainfolder+ '/'+file_class +'/'+ filename)
          pass
        
        pass 
      except:
        print('io erro')
        pass

In [ ]:
mkdir_for_keras('train')
mkdir_for_keras('valid')


In [ ]:
move2dir('jpg','train','valid',labels)

# 构建模型

In [3]:
class constants():
    def __init__(self):
      pass
# First time run, no unlocking
conv_base.trainable = False

# Let's construct that top layer replacement

x = conv_base.output
x = Flatten()(x)
x = Dropout(0.25)(x)
predictions = Dense(102,  kernel_initializer="glorot_uniform", activation='softmax')(x)
print('Stacking New Layers')
model = Model(inputs = conv_base.input, outputs=predictions)

Stacking New Layers


In [5]:


        
constants.NUM_CLASSES = 102
constants.BASE_DIR = ('.')
constants.SIZES = {
    'basic': 100
}
height = constants.SIZES['basic']
width = height
constants.STEPS_PER_EPOCH = 400
constants.TOTAL_EPOCHS = 10
constants.GENERATOR_BATCH_SIZE = 20
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from time import time
# Slow down training deeper into dataset
def schedule(epoch):
    if epoch < 6:
        # Warmup model first
        return .0000032
    elif epoch < 12:
        return .01
    elif epoch < 20:
        return .002
    elif epoch < 30:
        return .001
    elif epoch < 40:
        return .0004
    elif epoch < 60:
        return .00008
    elif epoch < 80:
        return .000016
    elif epoch < 95:
        return .0000032        
    else:
        return .0000009    
def make_callbacks(weights_file):
    # checkpoint
    filepath = weights_file
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

    # Update info
    tensorboard = TensorBoard(log_dir="log/{}".format(time()))

    # learning rate schedule
    lr_scheduler = LearningRateScheduler(schedule)

    # all the goodies
    return [lr_scheduler, checkpoint, tensorboard]
## callbacks_list = make_callbacks(weights_file)
import os
from keras.preprocessing.image import ImageDataGenerator
constants.BASE_DIR = ('.')
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=45,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     channel_shift_range=20,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    horizontal_flip=True,
    shear_range=0.5,
    zoom_range=0.3,
    fill_mode='wrap',
    width_shift_range=0.2,
    height_shift_range=0.2,
)

# Validation data should not be modified
validation_datagen = ImageDataGenerator(
    rescale=1./255
)

#train_dir = os.path.join(constants.BASE_DIR, 'data_train_clean')
#test_dir = os.path.join(constants.BASE_DIR, 'data_test')
train_dir = os.path.join(constants.BASE_DIR, 'train')
# train_dir = os.path.join(constants.BASE_DIR, 'moredata')
test_dir = os.path.join(constants.BASE_DIR, 'valid')
def create_generators(height, width):
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        class_mode='categorical',
        batch_size=constants.GENERATOR_BATCH_SIZE
    )

    validation_generator = validation_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        class_mode='categorical',
        batch_size=constants.GENERATOR_BATCH_SIZE
    )

    return[train_generator, validation_generator]
  

In [6]:
train_generator, validation_generator = create_generators(height, width)

Found 6509 images belonging to 102 classes.
Found 1680 images belonging to 102 classes.


In [7]:
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from time import time
# Slow down training deeper into dataset
def schedule(epoch):
    if epoch < 5:
        # Warmup model first
        return .0000032
    elif epoch < 12:
        return .0000032 #0.1
    elif epoch < 20:
        return .002
    elif epoch < 30:
        return .001
    elif epoch < 40:
        return .0004
    elif epoch < 60:
        return .00008
    elif epoch < 80:
        return .000016
    elif epoch < 95:
        return .0000032        
    else:
        return .0000009 
def make_callbacks(weights_file):
    # checkpoint
    filepath = weights_file
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

    # Update info
    tensorboard = TensorBoard(log_dir="log/{}".format(time()))

    # learning rate schedule
    lr_scheduler = LearningRateScheduler(schedule)

    # all the goodies
    return [lr_scheduler, checkpoint, tensorboard]
weights_file = 'checkpoint/best_weights.efficientnet-b4.hdf5'
callbacks_list = make_callbacks(weights_file)

# 训练和评估

In [ ]:
from keras.optimizers import SGD,Adam
print('Compile model')
# originally adam, but research says SGD with scheduler
opt = Adam(lr=0.01, amsgrad=True)
#opt = SGD(momentum=1,lr=1,nesterov=True) #,clipnorm=1. clipnorm=0.8
#opt = keras.optimizers.Adadelta( rho=0.95, epsilon=None, decay=0.0)
#opt = keras.optimizers.Nadam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
# lrate = 0.01 
# epoch = 10
# decay = lrate/epoch
# opt = SGD(lr=0.01, momentum=0.9, nesterov=True)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Get training/validation data via generators
train_generator, validation_generator = create_generators(height, width)
print('Start training!')
history = model.fit_generator(
    train_generator,
    callbacks=callbacks_list,
    epochs=50,
    steps_per_epoch=constants.STEPS_PER_EPOCH,
    shuffle=True,
    workers=4,
    use_multiprocessing=True,
    validation_data=validation_generator,
    validation_steps=50
)

# Save it for later
print('Saving Model')

model.save("/content/drive/My Drive/Kaggle/40EfficientNetB3_size." + str(width) + "x" + str(height) + ".hdf5")

400/400 [==============================] - 69s 169ms/step - loss: 3.1887 - accuracy: 0.2681 - val_loss: 2.8099 - val_accuracy: 0.3590
Epoch 11/50
400/400 [==============================] - 69s 169ms/step - loss: 3.0566 - accuracy: 0.2945 - val_loss: 2.7341 - val_accuracy: 0.3710
Epoch 12/50
400/400 [==============================] - 69s 169ms/step - loss: 3.0231 - accuracy: 0.2876 - val_loss: 2.6183 - val_accuracy: 0.3980
Epoch 13/50
400/400 [==============================] - 69s 169ms/step - loss: 7.3299 - accuracy: 0.2671 - val_loss: 6.6227 - val_accuracy: 0.5250
Epoch 14/50
400/400 [==============================] - 69s 169ms/step - loss: 7.2349 - accuracy: 0.4645 - val_loss: 6.0204 - val_accuracy: 0.5750
Epoch 15/50
400/400 [==============================] - 70s 170ms/step - loss: 6.1709 - accuracy: 0.5398 - val_loss: 4.9954 - val_accuracy: 0.6530
Epoch 16/50
400/400 [==============================] - 70s 171ms/step - loss: 5.7228 - accuracy: 0.5819 - val_loss: 4.7491 - val_accurac